In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import mxnet as mx

from joblib import Parallel, delayed
import time

/usr/local/lib/python2.7/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [7]:
patches_rootdir = '/home/yuncong/CSHL_data_patches/'
model_dir = '/home/yuncong/mxnet_models/'

features_rootdir = '/home/yuncong/CSHL_patch_features/'
if not os.path.exists(features_rootdir):
    os.makedirs(features_rootdir)

In [3]:
labels = ['BackG', '5N', '7n', '7N', '12N', 'Pn', 'VLL', 
          '6N', 'Amb', 'R', 'Tz', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

label_dict = dict([(l,i) for i, l in enumerate(labels)])

In [4]:
mean_img = mx.nd.load(os.path.join(model_dir, 'mean_224.nd'))['mean_img'].asnumpy()

In [5]:
progress_bar = FloatProgress(min=0, max=np.inf)

In [8]:
model_name = 'experiment0317'
model_iteration = 6

model = mx.model.FeedForward.load(os.path.join(model_dir, model_name), model_iteration, ctx=mx.gpu())

flatten_output = model.symbol.get_internals()['flatten_output']

model = mx.model.FeedForward(ctx=mx.gpu(), symbol=flatten_output, num_epoch=model_iteration,
                            arg_params=model.arg_params, aux_params=model.aux_params,
                            allow_extra_params=True)

In [13]:
# for stack in ['MD589', 'MD594', 'MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602']:
for stack in ['MD602']:

    dm = DataManager(stack=stack)

    table_filepath = os.path.join(patches_rootdir, '%(stack)s_indices_allROIs_allSections.h5'%{'stack':stack})
    indices_allROIs_allSections = pd.read_hdf(table_filepath, 'indices_allROIs_allSections')
    grid_parameters = pd.read_hdf(table_filepath, 'grid_parameters')
        
    patch_size, stride, w, h = grid_parameters.tolist()
    half_size = patch_size/2
    ys, xs = np.meshgrid(np.arange(half_size, h-half_size, stride), np.arange(half_size, w-half_size, stride),
                     indexing='xy')
    sample_locations = np.c_[xs.flat, ys.flat]
        
    first_bs_sec, last_bs_sec = section_range_lookup[stack]
    first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]

    progress_bar.min = first_detect_sec
    progress_bar.max = last_detect_sec
    display(progress_bar)
    
    for sec in range(first_detect_sec, last_detect_sec+1):
#     for sec in range(first_detect_sec, first_detect_sec+10):
#     for sec in range(first_detect_sec, first_detect_sec+1):
        
        if sec not in indices_allROIs_allSections.columns:
            continue

        progress_bar.value = sec

        dm.set_slice(sec)
        dm._load_image(['rgb-jpg'])
                
        indices_roi = indices_allROIs_allSections[sec]['roi1']

        n = len(indices_roi)
        print n, 'used samples'

        sample_locations_roi = sample_locations[indices_roi]

        patches2 = np.asarray([dm.image_rgb_jpg[y-half_size:y+half_size, x-half_size:x+half_size]
                              for x, y in sample_locations_roi])

        patches = np.rollaxis(patches2, 3, 1)
        patches = patches - mean_img
        

        batch_size = 512 # increasing to 892 does not save any time

        data_iter = mx.io.NDArrayIter(
            patches, 
            np.zeros((n, ), np.int),
            batch_size = batch_size,
            shuffle=False
        )

        t = time.time()

        features = model.predict(data_iter)
        
        features_dir = '/home/yuncong/CSHL_patch_features/%(stack)s/%(sec)04d' % {'stack': stack, 'sec': sec}
        if not os.path.exists(features_dir):
            os.makedirs(features_dir)

#         bp.pack_ndarray_file(features, features_dir + '/%(stack)s_%(sec)04d_roi1_features.bp'% {'stack': stack,
#                                                                                            'sec': sec})
        
        save_hdf(features, features_dir + '/%(stack)s_%(sec)04d_roi1_features.hdf'% {'stack': stack,
                                                                                           'sec': sec})
        
        sys.stderr.write('predict: %.2f seconds\n' % (time.time() - t))
        
        del patches, patches2, sample_locations_roi
        
    del sample_locations


22010 used samples


NameError: name 'save_hdf' is not defined